In [1]:
%pip install dash plotly
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output


Note: you may need to restart the kernel to use updated packages.


In [2]:
import os

# Read the airline data into pandas dataframe
if os.path.exists("spacex_launch_dash.csv"):
	spacex_df = pd.read_csv("spacex_launch_dash.csv")
else:
	# Create a sample dataframe if the file does not exist
	spacex_df = pd.DataFrame({
		'Payload Mass (kg)': [1000, 2000, 3000, 4000, 5000],
		'Launch Site': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40', 'CCAFS LC-40'],
		'class': [1, 0, 1, 1, 0]
	})

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Ensure dash is imported (in case previous cell was not run)
try:
    dash
except NameError:
    import dash

from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Ensure min_payload and max_payload are defined (in case previous cell was not run)
try:
    min_payload
    max_payload
except NameError:
    min_payload = 0
    max_payload = 10000

# Create a dash application
app = dash.Dash(__name__)       

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

# TASK 1: Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites
# dcc.Dropdown(id='site-dropdown',...)
                                   dcc.Dropdown(id='site-dropdown',
                                                  options=[
                                                  {'label': 'All Sites', 'value': 'ALL'},
                                                  {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                  {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                  {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                  {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                  ],
                                                  value='ALL',
                                                  placeholder="Select a Launch Site here",
                                                  searchable=True
                                                  ),
                                html.Br(),

# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                   html.Br(),

                                html.P("Payload range (Kg):"),
# TASK 3: Add a slider to select payload range
#dcc.RangeSlider(id='payload-slider',...)
                                   dcc.RangeSlider(id='payload-slider',
                                                 min=0, max=10000, step=1000,
                                                 marks={0: '0',
                                                           2500: '2500',
                                                           5000: '5000',
                                                           7500: '7500',
                                                           10000: '10000'},
                                                 value=[min_payload, max_payload]),
                                   html.Br(),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
# TASK 2: Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
                     names='Launch Site', 
                     title='Total Successful Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        success_counts = filtered_df['class'].value_counts().reset_index()
        success_counts.columns = ['class', 'count']
        fig = px.pie(success_counts, values='count', 
                     names='class', 
                     title=f'Total Launches for site {entered_site}')
        return fig   
# TASK 4: Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
                     Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload_range):
       low, high = payload_range
       mask = (spacex_df['Payload Mass (kg)'] >= low) & (spacex_df['Payload Mass (kg)'] <= high)
       filtered_df = spacex_df[mask]
       
       color_col = 'Booster Version Category' if 'Booster Version Category' in filtered_df.columns else None
       if entered_site == 'ALL':
              fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                             color=color_col,
                             title='Correlation between Payload and Success for all Sites')
              return fig
       else:
              site_df = filtered_df[filtered_df['Launch Site'] == entered_site]
              fig = px.scatter(site_df, x='Payload Mass (kg)', y='class',
                             color=color_col,
                             title=f'Correlation between Payload and Success for site {entered_site}')
              return fig
# Run the app
if __name__ == '__main__':
    app.run()